参考：《财务报表分析与证券估值（原书第5版）》 斯蒂芬·H·佩因曼 机械工业出版社

https://zhuanlan.zhihu.com/p/417274511

In [ ]:
import tushare as ts
import numpy as np
import pandas as pd
import time

# 获取利润表
def get_income(one_code,start_date,end_date):
    while True:
        try:
            one_income = pro.income(ts_code=one_code, start_date=start_date, end_date=end_date, report_type='1')
            break
        except Exception as exc:
            print(exc)
            time.sleep(5)
    return one_income

# 获取现金流量表
def get_cashflow(one_code,start_date,end_date):
    while True:
        try:
            one_cashflow= pro.cashflow(ts_code=one_code, start_date=start_date, end_date=end_date, report_type='1')
            break
        except Exception as exc:
            print(exc)
            time.sleep(5)
    return one_cashflow

# 获取资产负债表
def get_balancesheet(one_code,start_date,end_date):
    while True:
        try:
            one_balancesheet = pro.balancesheet(ts_code=one_code, start_date=start_date, end_date=end_date, report_type='1')
            break
        except Exception as exc:
            print(exc)
            time.sleep(5)
    return one_balancesheet

token = '此处输入你的token'
pro = ts.pro_api(token)
year_list = [str(i)+'1231' for i in range(2006,2021)] 
ts_code = input('>>>ts_code:')

'''
n_income:含少数股东净利润
fv_value_chg_gain：公允价值变动净收益
invest_income：投资净收益
ass_invest_income：对联营企业和合营企业投资收益
fin_exp：财务费用
oth_compr_income：其他综合收益
'''

one_income = get_income(ts_code,start_date='20061231',end_date='20211231')
one_income = one_income[one_income['end_date'].isin(year_list)] # 获取年报利润表数据
one_income = one_income.drop_duplicates(subset=['end_date'],keep='first') # 去重
one_income = one_income.reset_index(drop=True)
one_income = one_income.fillna(0)
one_income['金融费用'] = one_income.apply(lambda x: -x['fv_value_chg_gain']-x['invest_income']+x['ass_invest_income']+x['fin_exp'], axis=1)
one_income['经营利润'] = one_income.apply(lambda x: x['n_income'] + x['oth_compr_income'] + 0.75 * x['金融费用'], axis=1)
one_income = one_income.rename(columns={'n_income':'净利润','end_date':'年度'})

'''
total_hldr_eqy_inc_min_int:股东权益合计
minority_int:少数股东权益
total_assets:资产总计
total_liab:负债合计

金融资产项目：
money_cap:货币资金
trad_asset:交易性金融资产
int_receiv:应收利息
# nca_within_1y:一年内到期的非流动资产 # 忽略，可能低估金融资产
pur_resale_fa:买入返售金融资产
fa_avail_for_sale:可供出售金融资产
htm_invest:持有至到期投资
decr_in_disbur:发放贷款及垫款
deriv_assets:衍生金融资产

金融负债项目：
lt_borr:长期借款
st_borr:短期借款
trading_fl:交易性金融负债
int_payable:应付利息
st_bonds_payable:应付短期债券
non_cur_liab_due_1y:一年内到期的非流动负债
bond_payable:应付债券
deriv_liab:衍生金融负债
'''

financial_asset_list = ['money_cap','trad_asset','int_receiv','pur_resale_fa','fa_avail_for_sale','htm_invest','decr_in_disbur','deriv_assets']
financial_debt_list = ['lt_borr','st_borr','trading_fl','int_payable','st_bonds_payable','non_cur_liab_due_1y','bond_payable','deriv_liab']

one_balancesheet = get_balancesheet(ts_code,start_date='20061231',end_date='20211231')
one_balancesheet = one_balancesheet[one_balancesheet['end_date'].isin(year_list)]
one_balancesheet = one_balancesheet.drop_duplicates(subset=['end_date'],keep='first') # 去重
one_balancesheet = one_balancesheet.reset_index(drop=True)
one_balancesheet = one_balancesheet.fillna(0)
one_balancesheet['金融资产'] = one_balancesheet.apply(lambda x: x[financial_asset_list].sum(), axis=1)
one_balancesheet['金融负债'] = one_balancesheet.apply(lambda x: x[financial_debt_list].sum(), axis=1)
one_balancesheet['经营资产'] = one_balancesheet.apply(lambda x: x['total_assets'] - x['金融资产'], axis=1)
one_balancesheet['经营负债'] = one_balancesheet.apply(lambda x: x['total_liab'] - x['金融负债'], axis=1)
one_balancesheet['净金融负债'] = one_balancesheet.apply(lambda x: x['金融负债'] - x['金融资产'], axis=1)
one_balancesheet['净经营资产'] = one_balancesheet.apply(lambda x: x['经营资产'] - x['经营负债'], axis=1)
one_balancesheet = one_balancesheet.rename(columns={'end_date':'年度','minority_int':'少数股东权益','total_hldr_eqy_inc_min_int':'股东权益合计'})

table = pd.merge(one_income,one_balancesheet)
table[table.select_dtypes(include=['float64']).columns] /= 1e8 # 单位换算为亿元
table['剩余经营收益'] = table['经营利润'] - 0.1 * table['净经营资产'].shift(-1)
table['3年平均剩余收益'] = (table['剩余经营收益'] + table['剩余经营收益'].shift(-1) +\
                                table['剩余经营收益'].shift(-2)) / 3
table['无增长企业估值'] = table['净经营资产'] + table['3年平均剩余收益'] / 0.1
table['无增长企业市净率'] = table['无增长企业估值'] / table['股东权益合计']
table['无增长权益估值'] = table['无增长企业估值'] - table['净金融负债'] - table['少数股东权益'] * table['无增长企业市净率']
table['4%增长企业估值'] = table['净经营资产'] + table['3年平均剩余收益'] * 1.04 / 0.06
table['4%增长企业市净率'] = table['4%增长企业估值'] / table['股东权益合计']
table['4%增长权益估值'] = table['4%增长企业估值'] - table['净金融负债'] - table['少数股东权益'] * table['4%增长企业市净率']

table = table[['年度','净利润','金融费用','经营利润','剩余经营收益','3年平均剩余收益','少数股东权益','金融资产','金融负债',\
                 '经营资产','经营负债','净金融负债','净经营资产','无增长企业估值','无增长权益估值','4%增长企业估值','4%增长权益估值']]
print(table) 

# 生成excel表格
sheet_writer = pd.ExcelWriter(f'{ts_code}估值.xlsx')
table.to_excel(sheet_writer,index=False)
sheet_writer.save()